In [1]:
import numpy as np
import scipy.linalg
import scipy.special
import math
import sys
import matplotlib.pyplot as plt
import json

# Quartic EM Response Part 2: Calculate coefficients c_p

In this second notebook, we import the eigenvalues and eiegenvectors computed previously, and use them to calculate the current per orbital

### Coefficients

In [2]:
# These coefficients are defined in the tex document 'quartic EM Response' ('neat' version is best)

# n is in the definition of H^{2n}
# p is the prefactor to the coefficient c_p

# When dtilde is true (default), this uses David's definition of the Hamiltonians
def dcoeff(n, b, c, dtilde=True):
    if(dtilde):
        return (scipy.special.factorial(n-1)/scipy.special.factorial(b)/scipy.special.factorial(c) 
                * scipy.special.factorial2(2*n-1)/scipy.special.factorial2(2*n-b-c))
    else:
        return (2*scipy.special.factorial(n)/scipy.special.factorial(b)/scipy.special.factorial(c) 
                * scipy.special.factorial2(2*n-1)/scipy.special.factorial2(2*n-b-c))
    
def fcoeff(p, b, c):
    return (scipy.special.factorial(p)/scipy.special.factorial(b)/scipy.special.factorial(c)
            /scipy.special.factorial2(p-b-c))

def acoeff(t, b, c):
    if(t>=c):
        return np.sqrt(scipy.special.factorial(t)*scipy.special.factorial(t-c+b))/scipy.special.factorial(t-c)
    else:
        return 0
    
def bacterm(lam, b, c, t, coeff_matrix):
    if(b!=0):
        return b*acoeff(t,b-1,c)*np.conjugate(coeff_matrix[:,lam][t-c+b-1])
    else:
        return 0

def cacterm(lam, b, c, t, coeff_matrix):
    if(c!=0):
        return c*acoeff(t,b,c-1)*np.conjugate(coeff_matrix[:,lam][t-c+1+b])
    else:
        return 0



### Summand

In [3]:
# This is the main expression to calculate and sum over all indices

def summand(n, mu, lam, p, b, c, bet, gam, t, tau, bval, coeff_matrix, 
            energy_list, dtilde=True):
    coeff_matrix = np.array(coeff_matrix)
    energy_list = np.array(energy_list)
    return (-(bval**n) * (np.sqrt(2*bval) ** (-p-1)) * coeff_matrix[:,mu][t] *
            np.conjugate(coeff_matrix[:,lam][tau-gam+bet]) * coeff_matrix[:,mu][tau] *
            dcoeff(n, b, c, dtilde) * fcoeff(p, bet, gam) * acoeff(tau, bet, gam) /
            (energy_list[lam] - energy_list[mu]) * (bacterm(lam, b, c, t, coeff_matrix)
                                                    + cacterm(lam, b, c, t, coeff_matrix)))
            


### Index Lists

In [4]:
# These functions generate the lists of indices to be summed over

def mulist(lam, mu_max):
    list_out = np.array([j for j in range(mu_max+1)])
    list_out = np.delete(list_out, lam)
    return list_out

def tlist(t_max):
    return np.array([j for j in range(t_max+1)])

def betgamlist(p):
    list_out = []
    for b in range(p+1):
        for c in range(p+1-b):
            if(((b+c)%2)==(p%2)):
                list_out.append([b,c])
    return list_out

def bclist(n):
    list_out = []
    for b in range(2*n+1):
        for c in range(2*n+1-b):
            if(((b-c)%4)==0):
                list_out.append([b,c])
    return list_out
                


## Calculate Current Per Orbital
### Import Data From File

In [5]:
def read_evals_evecs(n, trunc_dim):
    
    filename_evals = "n"+str(n)+"_truncdim"+str(trunc_dim)+"_evals"
    filename_evecs = "n"+str(n)+"_truncdim"+str(trunc_dim)+"_evecs"
    
    with open(filename_evals) as f:
        q_evals = json.load(f)
        
    with open(filename_evecs) as f:
        q_evecs = json.load(f)
    
    return q_evals, q_evecs

### Calculate EM Response

In [58]:
def cp_coefficient_lambda(n, p, lam, coeff_matrix, energy_list, bval=1, mu_max=15, t_max=25, tol=1e-12, dtilde=True):
    
    this_tlist = tlist(t_max)
    this_betgamlist = betgamlist(p)
    this_bclist = bclist(n)
    this_mu_list = mulist(lam, mu_max)
    
    sum_total = 0
        
    for mu in this_mu_list:
        for tau in this_tlist:
            for t in this_tlist:
                for [b,c] in this_bclist:
                    for [bet, gam] in this_betgamlist:
                        this_element = summand(n, mu, lam, p, b, c, bet, gam, t, tau, bval,
                                                coeff_matrix, energy_list, dtilde=True)
                        if np.abs(this_element) > tol:
                            if np.abs(this_element) > 1e-2:
                                print('mu =', mu, ", t =",t, ", b =", b, ", c =", c, ", beta =", bet, ", gamma =", gam)
                                print(this_element)
                            sum_total += this_element
                            
    print('Total =', sum_total)
    return sum_total
    

### Check for Quartic

In [40]:
elist, coeffmat = read_evals_evecs(2, 50)

In [60]:
#p=1
out = cp_coefficient_lambda(2, 1, 0, coeffmat, elist)
print(out)
print(round(out,10))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5198047380842687
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.035787436744087624
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.013186641008677483
Total = 0.499999999980869
0.499999999980869
0.5


In [61]:
out = cp_coefficient_lambda(2, 1, 1, coeffmat, elist)
print(out)
print(round(out,10))

mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.5198047380842687
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.035787436744087624
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.01318664100867753
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5146767599693798
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.5146767599693794
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.06321452709561758
mu = 2 , t = 6 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.02717014610023477
mu = 4 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.013904778716822299
Total = 0.49999999939393
0.49999999939393
0.4999999994


In [63]:
out = cp_coefficient_lambda(2, 1, 2, coeffmat, elist)
print(out)
print(round(out,8))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.06321452709561756
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.027170146100234795
mu = 3 , t = 3 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5117037221703966
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
1.0234074443407932
mu = 3 , t = 3 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.09774873333131877
mu = 3 , t = 7 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.04812101027413798
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.016040336758045996
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.0104754430727036
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.024023952442793577
mu = 5 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
0.01032570087550407
Total = 0.49999999867084477
0.49999999867084477
0.5


In [64]:
#p=2
out = cp_coefficient_lambda(2, 2, 0, coeffmat, elist)
print(out)
print(round(out,10))

Total = 0
0
0


In [66]:
out = cp_coefficient_lambda(2, 2, 1, coeffmat, elist)
print(out)
print(round(out,10))

Total = 0
0
0


In [65]:
out = cp_coefficient_lambda(2, 2, 2, coeffmat, elist)
print(out)
print(round(out,10))

Total = 0
0
0


In [67]:
#p=3
out = cp_coefficient_lambda(2, 3, 0, coeffmat, elist)
print(out)
print(round(out,10))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.7797071071264031
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 3 , gamma = 0
-0.05368115511613144
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.05368115511613144
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.019779961513016222
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.019779961513016222
mu = 3 , t = 3 , b = 0 , c = 4 , beta = 0 , gamma = 3
0.08577759994904371
mu = 3 , t = 3 , b = 0 , c = 4 , beta = 2 , gamma = 1
-0.026575213600347748
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 3
-0.026575213600347748
Total = 0.7608540712060311
0.7608540712060311
0.7608540712


In [68]:
out = cp_coefficient_lambda(2, 3, 1, coeffmat, elist)
print(out)
print(round(out,10))

mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.7797071071264031
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.05368115511613143
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.019779961513016298
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 0 , gamma = 3
0.053681155116131446
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 2 , gamma = 1
-0.019779961513016225
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.7720151399540695
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.7720151399540695
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 3 , gamma = 0
-0.09482179064342632
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.7720151399540691
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 1 , gamma = 2
0.7720151399540691
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 3 , gamma = 0
-0.09482179064342627
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.09482179064342636
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 1 , gamma = 2
-0.09482179064342636
mu = 2 , t = 2 , b = 4 , c = 0 , 

In [69]:
out = cp_coefficient_lambda(2, 3, 2, coeffmat, elist)
print(out)
print(round(out,10))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.7720151399540699
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 2 , gamma = 1
-0.7720151399540699
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.7720151399540699
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 2 , gamma = 1
-0.7720151399540699
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.09482179064342634
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 2 , gamma = 1
0.09482179064342634
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.0407552191503522
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 2 , gamma = 1
-0.0407552191503522
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 3
0.09482179064342636
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 2 , gamma = 1
-0.0407552191503522
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 0 , gamma = 3
0.09482179064342636
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 2 , gamma = 1
-0.0407552191503522
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 0 , gamma = 3
-0.01164636742921997
mu = 3 , t = 3 , b = 1 , c = 1 , beta

In [70]:
out = cp_coefficient_lambda(2, 3, 3, coeffmat, elist)
print(out)
print(round(out,10))

mu = 0 , t = 0 , b = 4 , c = 0 , beta = 3 , gamma = 0
-0.08577759994904371
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 3 , gamma = 0
0.026575213600347744
mu = 0 , t = 0 , b = 4 , c = 0 , beta = 1 , gamma = 2
0.02657521360034774
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.767555583255595
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 2 , gamma = 1
-1.53511116651119
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 1 , gamma = 0
-1.5351111665111898
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 2 , gamma = 1
-3.0702223330223797
mu = 2 , t = 6 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.14662309999697812
mu = 2 , t = 6 , b = 0 , c = 4 , beta = 2 , gamma = 1
0.29324619999395624
mu = 2 , t = 6 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.012030252568534506
mu = 2 , t = 6 , b = 1 , c = 1 , beta = 2 , gamma = 1
-0.024060505137069012
mu = 2 , t = 6 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.07218151541120703
mu = 2 , t = 6 , b = 2 , c = 2 , beta = 2 , gamma = 1
-0.14436303082241406
mu = 2 , t = 2 , b = 1 , c = 1 , 

In [71]:
out = cp_coefficient_lambda(2, 3, 0, coeffmat, elist, mu_max=30, t_max=40)
print(out)
print(round(out,10))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.7797071071264031
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 3 , gamma = 0
-0.05368115511613144
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.05368115511613144
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.019779961513016222
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.019779961513016222
mu = 3 , t = 3 , b = 0 , c = 4 , beta = 0 , gamma = 3
0.08577759994904371
mu = 3 , t = 3 , b = 0 , c = 4 , beta = 2 , gamma = 1
-0.026575213600347748
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 3
-0.026575213600347748
Total = 0.7608540714873269
0.7608540714873269
0.7608540715


In [72]:
out = cp_coefficient_lambda(2, 3, 1, coeffmat, elist, mu_max=30, t_max=40)
print(out)
print(round(out,10))

mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.7797071071264031
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.05368115511613143
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.019779961513016298
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 0 , gamma = 3
0.053681155116131446
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 2 , gamma = 1
-0.019779961513016225
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.7720151399540695
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.7720151399540695
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 3 , gamma = 0
-0.09482179064342632
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.7720151399540691
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 1 , gamma = 2
0.7720151399540691
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 3 , gamma = 0
-0.09482179064342627
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.09482179064342636
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 1 , gamma = 2
-0.09482179064342636
mu = 2 , t = 2 , b = 4 , c = 0 , 

In [74]:
#p=4
out = cp_coefficient_lambda(2, 4, 0, coeffmat, elist)
print(out)
print(round(out,10))

Total = 0
0
0


In [75]:
out = cp_coefficient_lambda(2, 4, 1, coeffmat, elist)
print(out)
print(round(out,10))

Total = 0
0
0


In [76]:
#p=5
out = cp_coefficient_lambda(2, 5, 0, coeffmat, elist)
print(out)
print(round(out,10))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
1.949267767816007
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 3 , gamma = 0
-0.26840577558065715
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 4 , gamma = 1
-0.13420288779032857
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.13420288779032855
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 3 , gamma = 0
0.018479159598932572
mu = 1 , t = 5 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.012362475945635137
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.04944990378254055
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.012362475945635137
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 5
-0.09576654685752659
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.0988998075650811
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 2 , gamma = 3
0.0988998075650811
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 4 , gamma = 1
-0.016517433382289805
mu = 3 , t = 3 , b = 0 , c = 4 , beta = 0 , gamma = 3
0.4288879997452185
mu = 3 , t = 3 , b = 0 , c = 4 , b

In [77]:
out = cp_coefficient_lambda(2, 5, 1, coeffmat, elist)
print(out)
print(round(out,10))

mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-1.949267767816007
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 5 , gamma = 0
0.09576654685752659
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.13420288779032855
mu = 0 , t = 4 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.012362475945635196
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.04944990378254072
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 0 , gamma = 3
0.26840577558065715
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.012362475945635136
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 4
0.13420288779032857
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 2 , gamma = 1
-0.09889980756508109
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 3 , gamma = 2
-0.09889980756508109
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 0 , gamma = 3
-0.01847915959893257
mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 4
0.016517433382289805
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 0 , gamma = 1
1.9300378498851736
mu = 2 , t = 2 , b = 1 , c = 1 

In [78]:
out = cp_coefficient_lambda(2, 5, 2, coeffmat, elist)
print(out)
print(round(out,10))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 0
-1.9300378498851745
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 2 , gamma = 1
-3.860075699770349
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 5 , gamma = 0
0.22121248714890682
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 1 , gamma = 0
-1.9300378498851745
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 2 , gamma = 1
-3.860075699770349
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 5 , gamma = 0
0.22121248714890682
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.23705447660856577
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 2 , gamma = 1
0.47410895321713153
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 5 , gamma = 0
-0.027170146100234785
mu = 1 , t = 5 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.0203776095751761
mu = 1 , t = 5 , b = 1 , c = 1 , beta = 2 , gamma = 1
-0.0407552191503522
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.1018880478758805
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 2 , gamma = 1
-0.203776095751761
mu = 1 , t = 5 , b = 2 , c = 2 , beta =

mu = 5 , t = 9 , b = 0 , c = 4 , beta = 0 , gamma = 3
0.08685262421521174
mu = 5 , t = 9 , b = 0 , c = 4 , beta = 1 , gamma = 4
0.08685262421521174
mu = 5 , t = 9 , b = 0 , c = 4 , beta = 2 , gamma = 1
-0.037330003047348424
mu = 5 , t = 9 , b = 0 , c = 4 , beta = 3 , gamma = 2
-0.04977333739646456
mu = 5 , t = 9 , b = 2 , c = 2 , beta = 0 , gamma = 3
-0.038728640238068525
mu = 5 , t = 9 , b = 2 , c = 2 , beta = 1 , gamma = 4
-0.038728640238068525
mu = 5 , t = 9 , b = 2 , c = 2 , beta = 2 , gamma = 1
0.0166459018500623
mu = 5 , t = 9 , b = 2 , c = 2 , beta = 3 , gamma = 2
0.022194535800083064
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 0 , gamma = 3
0.08685262421521171
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 4
0.2605578726456352
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 2 , gamma = 1
-0.03872864023806851
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 3 , gamma = 2
-0.10327637396818269
mu = 5 , t = 5 , b = 1 , c = 1 , beta = 1 , gamma = 4
-0.022398001828409054
mu = 5 , t = 5 , b = 2 , c =

In [79]:
out = cp_coefficient_lambda(2, 5, 3, coeffmat, elist)
print(out)
print(round(out,10))

mu = 0 , t = 0 , b = 4 , c = 0 , beta = 3 , gamma = 0
-0.4288879997452186
mu = 0 , t = 4 , b = 1 , c = 1 , beta = 3 , gamma = 0
0.04429202266724623
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 3 , gamma = 0
0.13287606800173868
mu = 0 , t = 4 , b = 4 , c = 0 , beta = 3 , gamma = 0
-0.012113694214831739
mu = 0 , t = 0 , b = 4 , c = 0 , beta = 0 , gamma = 1
0.022146011333623115
mu = 0 , t = 0 , b = 4 , c = 0 , beta = 1 , gamma = 2
0.13287606800173868
mu = 0 , t = 0 , b = 4 , c = 0 , beta = 2 , gamma = 3
0.08858404533449246
mu = 0 , t = 0 , b = 4 , c = 0 , beta = 3 , gamma = 0
-0.01211369421483174
mu = 0 , t = 0 , b = 4 , c = 0 , beta = 4 , gamma = 1
-0.02422738842966348
mu = 0 , t = 4 , b = 1 , c = 1 , beta = 1 , gamma = 2
-0.013722346671773895
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 2
-0.041167040015321676
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 2 , gamma = 3
-0.027444693343547783
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 1 , gamma = 0
-1.9188889581389874
mu = 2 , t = 2 , b = 1 , c 

mu = 4 , t = 8 , b = 1 , c = 1 , beta = 0 , gamma = 5
-0.015241587716232552
mu = 4 , t = 8 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.012505543108523106
mu = 4 , t = 8 , b = 1 , c = 1 , beta = 2 , gamma = 3
0.02501108621704622
mu = 4 , t = 8 , b = 2 , c = 2 , beta = 0 , gamma = 5
-0.10669111401362785
mu = 4 , t = 8 , b = 2 , c = 2 , beta = 1 , gamma = 2
0.08753880175966174
mu = 4 , t = 8 , b = 2 , c = 2 , beta = 2 , gamma = 3
0.1750776035193235
mu = 4 , t = 8 , b = 2 , c = 2 , beta = 4 , gamma = 1
-0.02362876054052318
mu = 4 , t = 8 , b = 4 , c = 0 , beta = 2 , gamma = 3
-0.011814380270261599
mu = 4 , t = 4 , b = 1 , c = 1 , beta = 0 , gamma = 5
-0.0411658088472918
mu = 4 , t = 4 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.01944531779364846
mu = 4 , t = 4 , b = 1 , c = 1 , beta = 2 , gamma = 3
0.0648177259788282
mu = 4 , t = 4 , b = 2 , c = 2 , beta = 0 , gamma = 5
-0.12349742654187536
mu = 4 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 2
0.05833595338094537
mu = 4 , t = 4 , b = 2 , c = 2 ,

mu = 8 , t = 8 , b = 4 , c = 0 , beta = 0 , gamma = 5
0.0497361588444704
mu = 8 , t = 8 , b = 4 , c = 0 , beta = 1 , gamma = 2
-0.040807932222143795
mu = 8 , t = 8 , b = 4 , c = 0 , beta = 2 , gamma = 3
-0.0816158644442876
mu = 8 , t = 8 , b = 4 , c = 0 , beta = 4 , gamma = 1
0.011015010935130964
mu = 8 , t = 12 , b = 2 , c = 2 , beta = 0 , gamma = 5
-0.04059723381400893
mu = 8 , t = 12 , b = 2 , c = 2 , beta = 1 , gamma = 2
0.033309551931206056
mu = 8 , t = 12 , b = 2 , c = 2 , beta = 2 , gamma = 3
0.06661910386241213
mu = 8 , t = 4 , b = 1 , c = 1 , beta = 2 , gamma = 3
-0.013846989712870432
mu = 8 , t = 4 , b = 2 , c = 2 , beta = 0 , gamma = 5
0.026382715053130522
mu = 8 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 2
-0.012462290741583384
mu = 8 , t = 4 , b = 2 , c = 2 , beta = 2 , gamma = 3
-0.041540969138611286
mu = 8 , t = 8 , b = 1 , c = 1 , beta = 0 , gamma = 5
-0.010073777599171399
mu = 8 , t = 8 , b = 1 , c = 1 , beta = 2 , gamma = 3
0.015861691395812477
mu = 8 , t = 8 , b = 

In [80]:
# Extra Check
# 1.745910279
out = cp_coefficient_lambda(2, 5, 0, coeffmat, elist, mu_max=30, t_max=40)
print(out)
print(round(out,10))

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
1.949267767816007
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 3 , gamma = 0
-0.26840577558065715
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 4 , gamma = 1
-0.13420288779032857
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.13420288779032855
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 3 , gamma = 0
0.018479159598932572
mu = 1 , t = 5 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.012362475945635137
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.04944990378254055
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.012362475945635137
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 5
-0.09576654685752659
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 2
0.0988998075650811
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 2 , gamma = 3
0.0988998075650811
mu = 1 , t = 1 , b = 1 , c = 1 , beta = 4 , gamma = 1
-0.016517433382289805
mu = 3 , t = 3 , b = 0 , c = 4 , beta = 0 , gamma = 3
0.4288879997452185
mu = 3 , t = 3 , b = 0 , c = 4 , b

### Old Check for Quartic

In [46]:
#p=1
cp_coefficient_lambda(2, 1, 0, coeffmat, elist)

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5198047380842687
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.035787436744087624
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.013186641008677483
Total = 0.49999966969909093


0.49999966969909093

In [47]:
cp_coefficient_lambda(2, 1, 0, coeffmat, elist, mu_max=10, t_max=20)

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5198047380842687
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.035787436744087624
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.013186641008677483
Total = 0.4999999927664726


0.4999999927664726

In [53]:
cp_coefficient_lambda(2, 1, 0, coeffmat, elist, mu_max=15, t_max=25)

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5198047380842687
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.035787436744087624
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.013186641008677483
Total = 0.4999999994923225


0.4999999994923225

In [49]:
cp_coefficient_lambda(2, 1, 0, coeffmat, elist, mu_max=30, t_max=40)

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5198047380842687
mu = 1 , t = 1 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.035787436744087624
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.013186641008677483
Total = 0.4999999988772629


0.4999999988772629

In [50]:
cp_coefficient_lambda(2, 1, 1, coeffmat, elist)

mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.5198047380842687
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.035787436744087624
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.01318664100867753
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5146767599693798
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.5146767599693794
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.06321452709561758
mu = 2 , t = 6 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.02717014610023477
mu = 4 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.013904778716822299
Total = 0.4999991674946843


0.4999991674946843

In [52]:
cp_coefficient_lambda(2, 1, 1, coeffmat, elist, mu_max=15, t_max=25)

mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.5198047380842687
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.035787436744087624
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.01318664100867753
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5146767599693798
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.5146767599693794
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.06321452709561758
mu = 2 , t = 6 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.02717014610023477
mu = 4 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.013904778716822299
Total = 0.4999999987103566


0.4999999987103566

In [56]:
cp_coefficient_lambda(2, 1, 1, coeffmat, elist, mu_max=30, t_max=40)

mu = 0 , t = 0 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.5198047380842687
mu = 0 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.035787436744087624
mu = 0 , t = 4 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.01318664100867753
mu = 2 , t = 2 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5146767599693798
mu = 2 , t = 2 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.5146767599693794
mu = 2 , t = 2 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.06321452709561758
mu = 2 , t = 6 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.02717014610023477
mu = 4 , t = 4 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.013904778716822299
Total = 0.4999999986696898


0.4999999986696898

In [54]:
cp_coefficient_lambda(2, 1, 2, coeffmat, elist)

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.06321452709561756
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.027170146100234795
mu = 3 , t = 3 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5117037221703966
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
1.0234074443407932
mu = 3 , t = 3 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.09774873333131877
mu = 3 , t = 7 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.04812101027413798
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.016040336758045996
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.0104754430727036
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.024023952442793577
mu = 5 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
0.01032570087550407
Total = 0.499997131676451


0.499997131676451

In [55]:
cp_coefficient_lambda(2, 1, 2, coeffmat, elist, mu_max=15, t_max=25)

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.06321452709561756
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.027170146100234795
mu = 3 , t = 3 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5117037221703966
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
1.0234074443407932
mu = 3 , t = 3 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.09774873333131877
mu = 3 , t = 7 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.04812101027413798
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.016040336758045996
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.0104754430727036
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.024023952442793577
mu = 5 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
0.01032570087550407
Total = 0.49999999855253097


0.49999999855253097

In [57]:
cp_coefficient_lambda(2, 1, 2, coeffmat, elist, mu_max=30, t_max=40)

mu = 1 , t = 1 , b = 1 , c = 1 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 1 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.51467675996938
mu = 1 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.06321452709561756
mu = 1 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
-0.027170146100234795
mu = 3 , t = 3 , b = 1 , c = 1 , beta = 0 , gamma = 1
0.5117037221703966
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
1.0234074443407932
mu = 3 , t = 3 , b = 4 , c = 0 , beta = 0 , gamma = 1
-0.09774873333131877
mu = 3 , t = 7 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.04812101027413798
mu = 3 , t = 3 , b = 2 , c = 2 , beta = 0 , gamma = 1
0.016040336758045996
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
0.0104754430727036
mu = 5 , t = 5 , b = 0 , c = 4 , beta = 1 , gamma = 0
-0.024023952442793577
mu = 5 , t = 5 , b = 2 , c = 2 , beta = 1 , gamma = 0
0.01032570087550407
Total = 0.4999999997024069


0.4999999997024069